# YouTube Video Transcription Pipeline
## Optimized for Kaggle - No OOM Errors

This notebook:
- Downloads YouTube videos
- Extracts audio
- Transcribes using OpenAI Whisper (memory-efficient)
- Creates timestamped utterances
- Exports JSON output

In [ ]:
# ====================================================================
# CELL 1: Install Dependencies
# ====================================================================
print("📦 Installing dependencies...")
!pip install -q yt-dlp openai-whisper
print("✅ Dependencies installed!")

In [ ]:
# ====================================================================
# CELL 2: Import Libraries
# ====================================================================
import os
import json
import subprocess
import gc
from pathlib import Path
import whisper
import torch

print(f"🖥️  Device: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    # Clear any existing cache
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# ====================================================================
# CELL 3: Configuration
# ====================================================================
# ⚠️ CHANGE THIS to your YouTube video URL
VIDEO_URL = "https://www.youtube.com/watch?v=XNQTWZ87K4I"  # Replace with your video

# Model size options (smaller = less memory):
# "tiny"   - ~1GB VRAM, fastest, least accurate
# "base"   - ~1GB VRAM, fast, decent accuracy  ✅ RECOMMENDED for Kaggle
# "small"  - ~2GB VRAM, good accuracy
# "medium" - ~5GB VRAM, better accuracy
# "large"  - ~10GB VRAM, best accuracy (may OOM on Kaggle)

CONFIG = {
    "model_size": "base",  # Change to "small" or "medium" if you have enough memory
    "language": "en",      # "en" for English, "es" for Spanish, None for auto-detect
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "output_dir": "/kaggle/working/output"
}

# Create directories
os.makedirs("/kaggle/working/videos", exist_ok=True)
os.makedirs("/kaggle/working/audio", exist_ok=True)
os.makedirs(CONFIG["output_dir"], exist_ok=True)

print("✅ Configuration loaded")
print(f"   Model: {CONFIG['model_size']}")
print(f"   Language: {CONFIG['language'] or 'auto-detect'}")
print(f"   Device: {CONFIG['device']}")

In [ ]:
# ====================================================================
# CELL 4: Download Video
# ====================================================================
def download_youtube_video(url: str):
    """Download YouTube video using yt-dlp."""
    print(f"📥 Downloading video from: {url}")
    
    import yt_dlp
    
    output_dir = "/kaggle/working/videos"
    
    ydl_opts = {
        'format': 'best[ext=mp4]/best',
        'outtmpl': os.path.join(output_dir, '%(id)s.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        video_id = info['id']
        video_title = info.get('title', 'Unknown')
        duration = info.get('duration', 0)
        video_path = os.path.join(output_dir, f"{video_id}.mp4")
    
    print(f"✅ Video downloaded: {video_title}")
    print(f"   Duration: {duration//60}m {duration%60}s")
    return video_path, video_id, video_title, duration

# Download the video
video_path, video_id, video_title, video_duration = download_youtube_video(VIDEO_URL)

In [ ]:
# ====================================================================
# CELL 5: Extract Audio
# ====================================================================
def extract_audio(video_path: str) -> str:
    """Extract audio from video using ffmpeg."""
    print(f"🎵 Extracting audio from: {Path(video_path).name}")
    
    video_name = Path(video_path).stem
    audio_path = f"/kaggle/working/audio/{video_name}.mp3"
    
    # Extract as MP3 (Whisper handles this well)
    cmd = [
        'ffmpeg', '-i', video_path,
        '-vn', '-acodec', 'libmp3lame',
        '-ar', '16000', '-ac', '1',
        '-b:a', '64k',
        '-y', audio_path
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception(f"FFmpeg error: {result.stderr}")
    
    file_size = os.path.getsize(audio_path) / (1024*1024)
    print(f"✅ Audio extracted: {file_size:.2f} MB")
    return audio_path

# Extract audio
audio_path = extract_audio(video_path)

In [ ]:
# ====================================================================
# CELL 6: Transcribe Audio with Whisper
# ====================================================================
def transcribe_with_whisper(audio_path: str, model_size: str, language: str = None):
    """Transcribe audio using Whisper (memory-efficient)."""
    print(f"🎤 Loading Whisper model: {model_size}")
    
    # Load model with FP16 if on GPU (saves memory)
    model = whisper.load_model(
        model_size,
        device=CONFIG['device']
    )
    
    print(f"🎯 Transcribing audio...")
    print("   This may take a few minutes depending on video length")
    
    # Transcribe with options
    result = model.transcribe(
        audio_path,
        language=language,
        fp16=torch.cuda.is_available(),  # Use FP16 on GPU (faster, less memory)
        verbose=False,
        word_timestamps=True  # Get word-level timestamps
    )
    
    # Clean up model to free memory
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    print(f"✅ Transcription complete!")
    print(f"   Detected language: {result['language']}")
    print(f"   Segments: {len(result['segments'])}")
    
    return result

# Transcribe
transcription = transcribe_with_whisper(
    audio_path,
    CONFIG['model_size'],
    CONFIG['language']
)

In [ ]:
# ====================================================================
# CELL 7: Process Results
# ====================================================================
def create_utterances(transcription_result):
    """Convert Whisper segments to utterance format."""
    utterances = []
    
    for segment in transcription_result['segments']:
        utterances.append({
            "text": segment['text'].strip(),
            "start_ms": int(segment['start'] * 1000),
            "end_ms": int(segment['end'] * 1000),
            "confidence": segment.get('confidence', 0.0),
            "speaker": "default",
            "words": segment.get('words', [])  # Word-level timestamps if available
        })
    
    return utterances

# Create utterances
utterances = create_utterances(transcription)
full_transcript = transcription['text']

print(f"✅ Created {len(utterances)} utterances")
print(f"\n📝 Transcript preview (first 500 chars):")
print("="*70)
print(full_transcript[:500] + "..." if len(full_transcript) > 500 else full_transcript)
print("="*70)

In [ ]:
# ====================================================================
# CELL 8: Save Results
# ====================================================================
# Prepare final output
result = {
    "video_id": video_id,
    "video_title": video_title,
    "video_url": VIDEO_URL,
    "video_path": video_path,
    "audio_path": audio_path,
    "duration_ms": video_duration * 1000,
    "full_transcript": full_transcript,
    "utterances": utterances,
    "utterance_count": len(utterances),
    "model_used": f"whisper-{CONFIG['model_size']}",
    "language": transcription['language']
}

# Save to JSON
output_file = f"{CONFIG['output_dir']}/transcript_{video_id}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Also save just the text
text_file = f"{CONFIG['output_dir']}/transcript_{video_id}.txt"
with open(text_file, 'w', encoding='utf-8') as f:
    f.write(full_transcript)

file_size = os.path.getsize(output_file) / (1024*1024)

print("\n" + "="*70)
print("✅ TRANSCRIPTION COMPLETE!")
print("="*70)
print(f"📊 Results:")
print(f"   - Video: {video_title}")
print(f"   - Duration: {video_duration//60}m {video_duration%60}s")
print(f"   - Language: {transcription['language']}")
print(f"   - Utterances: {len(utterances)}")
print(f"   - Transcript length: {len(full_transcript)} characters")
print(f"\n💾 Files saved:")
print(f"   - JSON: {output_file} ({file_size:.2f} MB)")
print(f"   - Text: {text_file}")
print("="*70)

In [ ]:
# ====================================================================
# CELL 9: Display Sample Results
# ====================================================================
print("\n📄 First 5 utterances:")
print("="*70)
for i, utt in enumerate(utterances[:5], 1):
    start_time = utt['start_ms'] / 1000
    end_time = utt['end_ms'] / 1000
    print(f"\n[{i}] {start_time:.1f}s - {end_time:.1f}s")
    print(f"    {utt['text']}")
    if utt.get('confidence'):
        print(f"    Confidence: {utt['confidence']:.2f}")

print("\n" + "="*70)
print("✅ All done! Download your files from /kaggle/working/output/")
print("="*70)

**Phase 2**

In [ ]:
# ====================================================================
# CELL 1: Install Dependencies
# ====================================================================
print("📦 Installing dependencies...")
!pip install -q sentence-transformers torch numpy
print("✅ Dependencies installed!")

In [ ]:
# ====================================================================
# CELL 2: Import Libraries
# ====================================================================
import os
import json
import re
from typing import List, Dict, Any, Tuple
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
import gc

print(f"🖥️  Device: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# ====================================================================
# CELL 3: Configuration
# ====================================================================

CONFIG = {
    "phase1_file": "/kaggle/working/output/transcript_XNQTWZ87K4I.json", 
    "use_bert": True,
    "bert_model": "all-MiniLM-L6-v2",
    "similarity_threshold": 0.38,  # Slightly lower to catch semantic variations
    
    "reference_phrases": [
        # Navigation/Showing Actions
        "take a look on",
        "as you see right here",
        "show you how it can be done",
        "if you take a look",
        "if I go to",
        "press refresh",
        "inside the administration panel",
        
        # Specific Visual Objects in Video
        "cookie banner",
        "detailed cookie banner",
        "little widget",
        "trash bag",
        "consent tracking",
        "record of the consents",
        
        # Positional/Structural references
        "down here",
        "inside our back-end",
        "second part is",
        "third point is"
    ],
    "output_dir": "/kaggle/working/output"
}

os.makedirs(CONFIG['output_dir'], exist_ok=True)
print("✅ Configuration loaded")
print(f"   BERT: {'Enabled' if CONFIG['use_bert'] else 'Disabled'}")
print(f"   Similarity threshold: {CONFIG['similarity_threshold']}")
print(f"   Reference phrases: {len(CONFIG['reference_phrases'])}")

In [ ]:
# ====================================================================
# CELL 4: Load Phase 1 Output
# ====================================================================

print(f"📂 Loading Phase 1 transcript from: {CONFIG['phase1_file']}")

with open(CONFIG['phase1_file'], 'r', encoding='utf-8') as f:
    phase1_data = json.load(f)

utterances = phase1_data['utterances']
video_id = phase1_data['video_id']

print(f"✅ Loaded transcript:")
print(f"   Video ID: {video_id}")
print(f"   Utterances: {len(utterances)}")
print(f"   Duration: {phase1_data['duration_ms']/1000:.1f}s")
print(f"\n📝 First utterance: {utterances[0]['text'][:100]}...")

In [ ]:
# ====================================================================
# CELL 5: Reference Detector Class
# ====================================================================

class EnglishReferenceDetector:
    """Detect visual references in English transcripts."""
    
    def __init__(self, phrases: List[str], model_name: str = None,
                 use_bert: bool = True, threshold: float = 0.38):
        self.phrases = phrases
        self.use_bert = use_bert
        self.threshold = threshold
        self.model = None
        
        self.patterns = [
            re.compile(r'\b' + re.escape(phrase) + r'\b', re.IGNORECASE)
            for phrase in phrases
        ]
        
        if use_bert and model_name:
            print(f"🤖 Loading semantic model: {model_name}")
            device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model = SentenceTransformer(model_name, device=device)
            print(f"✅ Model loaded on {device}")
    
    def detect_with_regex(self, utterances: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Detect references using regex pattern matching."""
        print("🔍 Detecting with regex patterns...")
        references = []
        reference_id = 1
        
        for utt_idx, utterance in enumerate(utterances):
            text = utterance['text']
            
            for phrase, pattern in zip(self.phrases, self.patterns):
                match = pattern.search(text)
                
                if match:
                    references.append({
                        "reference_id": f"REF_{reference_id:03d}",
                        "text": text,
                        "timestamp_ms": utterance['start_ms'],
                        "end_ms": utterance['end_ms'],
                        "matched_phrase": phrase,
                        "reference_type": self._classify_type(phrase),
                        "detection_method": "regex",
                        "confidence": 0.95,
                        "context_before": self._get_context(utterances, utt_idx, -1),
                        "context_after": self._get_context(utterances, utt_idx, 1)
                    })
                    reference_id += 1
                    break  # Only count once per utterance
        
        print(f"   Found {len(references)} regex matches")
        return references
    
    def detect_with_semantic(self, utterances: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Detect references using semantic similarity."""
        if not self.model:
            print("⚠️  Semantic model not loaded, skipping")
            return []
        
        print(f"🧠 Detecting with semantic matching (threshold={self.threshold})...")
        references = []
        reference_id = 1000  # Different ID range for semantic
        
        # Encode reference phrases
        print("   Encoding reference phrases...")
        reference_embeddings = self.model.encode(
            self.phrases,
            convert_to_tensor=True,
            show_progress_bar=False
        )
        
        # Process in batches to save memory
        batch_size = 32
        print(f"   Processing {len(utterances)} utterances in batches...")
        
        for batch_start in range(0, len(utterances), batch_size):
            batch_end = min(batch_start + batch_size, len(utterances))
            batch = utterances[batch_start:batch_end]
            
            # Encode batch
            texts = [u['text'] for u in batch]
            text_embeddings = self.model.encode(
                texts,
                convert_to_tensor=True,
                show_progress_bar=False
            )
            
            # Calculate similarities
            similarities = torch.mm(text_embeddings, reference_embeddings.T)
            max_sims, _ = similarities.max(dim=1)
            
            # Find matches
            for i, (utterance, sim) in enumerate(zip(batch, max_sims)):
                if sim.item() >= self.threshold:
                    utt_idx = batch_start + i
                    references.append({
                        "reference_id": f"SEM_{reference_id:03d}",
                        "text": utterance['text'],
                        "timestamp_ms": utterance['start_ms'],
                        "end_ms": utterance['end_ms'],
                        "matched_phrase": "semantic_match",
                        "reference_type": "contextual",
                        "detection_method": "semantic",
                        "confidence": float(sim.item()),
                        "context_before": self._get_context(utterances, utt_idx, -1),
                        "context_after": self._get_context(utterances, utt_idx, 1)
                    })
                    reference_id += 1
        
        # Clean up
        del reference_embeddings, text_embeddings
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        print(f"   Found {len(references)} semantic matches")
        return references
    
    def _classify_type(self, phrase: str) -> str:
        """Classify reference type based on software demonstration context."""
        phrase_lower = phrase.lower()
        
        # New Categories for UI/Software Walkthroughs
        if any(word in phrase_lower for word in ['banner', 'widget', 'trash bag', 'button']):
            return 'ui_element'
        elif any(word in phrase_lower for word in ['panel', 'back-end', 'screen', 'go to']):
            return 'interface_navigation'
        elif any(word in phrase_lower for word in ['show you', 'look at', 'see right here']):
            return 'visual_demonstration'
        elif any(word in phrase_lower for word in ['part', 'point', 'step']):
            return 'structural_transition'
        else:
            return 'general_reference'
    
    def _get_context(self, utterances: List[Dict], index: int, offset: int) -> str:
        target_idx = index + offset
        if 0 <= target_idx < len(utterances):
            return utterances[target_idx]['text']
        return ""

print("✅ Detector class defined")

In [ ]:
# ====================================================================
# CELL 6: Run Detection
# ====================================================================

print("="*70)
print("🚀 STARTING REFERENCE DETECTION")
print("="*70)

# Initialize detector
detector = EnglishReferenceDetector(
    phrases=CONFIG['reference_phrases'],
    model_name=CONFIG['bert_model'] if CONFIG['use_bert'] else None,
    use_bert=CONFIG['use_bert'],
    threshold=CONFIG['similarity_threshold']
)

# Step 1: Regex detection (fast, high precision)
print("\n[1/3] Regex pattern matching...")
regex_refs = detector.detect_with_regex(utterances)

# Step 2: Semantic detection (slower, catches more)
semantic_refs = []
if CONFIG['use_bert']:
    print("\n[2/3] Semantic matching...")
    semantic_refs = detector.detect_with_semantic(utterances)

# Step 3: Merge and deduplicate
print("\n[3/3] Merging results...")
all_refs = regex_refs + semantic_refs

# Remove duplicates (within 2 second window)
unique_refs = []
seen_windows = set()

for ref in sorted(all_refs, key=lambda x: (x['timestamp_ms'], -x['confidence'])):
    # Use 2-second windows
    window = ref['timestamp_ms'] // 2000
    
    if window not in seen_windows:
        unique_refs.append(ref)
        seen_windows.add(window)

print(f"\n✅ Detection complete!")
print(f"   Regex matches: {len(regex_refs)}")
print(f"   Semantic matches: {len(semantic_refs)}")
print(f"   Unique references: {len(unique_refs)}")

In [ ]:
# ====================================================================
# CELL 7: Save Results
# ====================================================================

result = {
    "video_id": video_id,
    "video_url": phase1_data.get('video_url', ''),
    "references": unique_refs,
    "reference_count": len(unique_refs),
    "detection_summary": {
        "regex_count": len(regex_refs),
        "semantic_count": len(semantic_refs),
        "unique_count": len(unique_refs),
        "method": "regex+semantic" if CONFIG['use_bert'] else "regex_only"
    },
    "config": {
        "model": CONFIG['bert_model'] if CONFIG['use_bert'] else None,
        "threshold": CONFIG['similarity_threshold'],
        "phrase_count": len(CONFIG['reference_phrases'])
    }
}

# Save JSON
output_file = f"{CONFIG['output_dir']}/phase2_references_{video_id}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Save simple text report
report_file = f"{CONFIG['output_dir']}/phase2_report_{video_id}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(f"Visual Reference Detection Report\n")
    f.write(f"Video ID: {video_id}\n")
    f.write(f"="*70 + "\n\n")
    
    for i, ref in enumerate(unique_refs, 1):
        time_sec = ref['timestamp_ms'] / 1000
        f.write(f"[{i}] {time_sec:.1f}s - {ref['detection_method']} (conf: {ref['confidence']:.2f})\n")
        f.write(f"    {ref['text']}\n\n")

file_size = os.path.getsize(output_file) / 1024

print("\n" + "="*70)
print("✅ PHASE 2 COMPLETE!")
print("="*70)
print(f"📊 Results:")
print(f"   - Total references: {len(unique_refs)}")
print(f"   - Average confidence: {np.mean([r['confidence'] for r in unique_refs]):.2f}")
print(f"\n💾 Files saved:")
print(f"   - JSON: {output_file} ({file_size:.1f} KB)")
print(f"   - Report: {report_file}")
print("="*70)

In [ ]:
# ====================================================================
# CELL 8: Display Sample Results
# ====================================================================

print("\n📋 Sample References (first 10):")
print("="*70)

for i, ref in enumerate(unique_refs[:10], 1):
    time = ref['timestamp_ms'] / 1000
    method = ref['detection_method']
    conf = ref['confidence']
    
    print(f"\n[{i}] {time:.1f}s | {method} | confidence: {conf:.2f}")
    print(f"    Type: {ref['reference_type']}")
    print(f"    Text: {ref['text'][:120]}...")
    
    if ref.get('context_before'):
        print(f"    Before: {ref['context_before'][:80]}...")

if len(unique_refs) > 10:
    print(f"\n... and {len(unique_refs) - 10} more references")

print("\n" + "="*70)

In [ ]:
# ====================================================================
# CELL 9: Statistics & Visualization
# ====================================================================

import matplotlib.pyplot as plt

if len(unique_refs) > 0:
    # Reference timeline
    timestamps = [r['timestamp_ms'] / 1000 for r in unique_refs]
    confidences = [r['confidence'] for r in unique_refs]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    # Timeline scatter plot
    colors = ['blue' if r['detection_method'] == 'regex' else 'orange' for r in unique_refs]
    ax1.scatter(timestamps, confidences, c=colors, alpha=0.6, s=100)
    ax1.axhline(y=CONFIG['similarity_threshold'], color='red', linestyle='--', alpha=0.5, label='Threshold')
    ax1.set_xlabel('Time (seconds)', fontsize=12)
    ax1.set_ylabel('Confidence', fontsize=12)
    ax1.set_title('Visual References Timeline', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(['Threshold', 'Regex', 'Semantic'])
    
    # Type distribution
    types = [r['reference_type'] for r in unique_refs]
    type_counts = {t: types.count(t) for t in set(types)}
    ax2.bar(type_counts.keys(), type_counts.values(), color='steelblue')
    ax2.set_xlabel('Reference Type', fontsize=12)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Reference Type Distribution', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/phase2_analysis_{video_id}.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print("📊 Visualization saved!")
else:
    print("⚠️  No references found to visualize")

# Summary stats
print(f"\n📈 Statistics:")
print(f"   Average time between refs: {np.mean(np.diff(timestamps)) if len(timestamps) > 1 else 0:.1f}s")
print(f"   Reference density: {len(unique_refs) / (phase1_data['duration_ms']/1000) * 60:.1f} refs/minute")

print("\n✅ All done! Download files from /kaggle/working/output/")

**Phase 3**

In [ ]:
# ====================================================================
# CELL 1: Install Dependencies
# ====================================================================
print("📦 Installing dependencies...")

# Core dependencies (always needed)
!pip install -q opencv-python pillow imagehash

# # Optional: OCR (uncomment if you want text extraction)
# !pip install -q pytesseract
# !apt-get install -y tesseract-ocr

# Optional: Better OCR for equations/diagrams
!pip install -q paddlepaddle paddleocr

# Optional: Google Gemini for AI descriptions
!pip install -q google-generativeai

print("✅ Dependencies installed!")

In [ ]:
# ====================================================================
# CELL 2: Import Libraries
# ====================================================================
import os
import json
import cv2
import numpy as np
from PIL import Image
import imagehash
from pathlib import Path
from typing import Dict, List, Any, Tuple, Optional
import shutil

# Optional imports (will fail gracefully if not installed)
try:
    import pytesseract
    TESSERACT_AVAILABLE = True
except:
    TESSERACT_AVAILABLE = False
    print("⚠️  Tesseract not available (OCR disabled)")

try:
    from paddleocr import PaddleOCR
    PADDLE_AVAILABLE = True
except:
    PADDLE_AVAILABLE = False
    print("⚠️  PaddleOCR not available")

try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except:
    GEMINI_AVAILABLE = False
    print("⚠️  Google Gemini not available")

print("✅ Libraries imported")
print(f"   OCR (Tesseract): {'✓' if TESSERACT_AVAILABLE else '✗'}")
print(f"   OCR (Paddle): {'✓' if PADDLE_AVAILABLE else '✗'}")
print(f"   AI (Gemini): {'✓' if GEMINI_AVAILABLE else '✗'}")

In [ ]:
# ====================================================================
# CELL 3: Configuration
# ====================================================================

import os
from kaggle_secrets import UserSecretsClient

# Load Gemini API key from Kaggle secrets
user_secrets = UserSecretsClient()
try:
    gemini_api_key = user_secrets.get_secret("GEMINI_API_KEY")
    print("✅ Gemini API key loaded from Kaggle secrets")
except:
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if gemini_api_key:
        print("✅ Gemini API key loaded from environment variable")
    else:
        gemini_api_key = None
        print("⚠️  GEMINI_API_KEY not found in Kaggle secrets")

CONFIG = {
    # Input files
    "phase1_file": "/kaggle/working/output/transcript_XNQTWZ87K4I.json",  # ⚠️ UPDATE
    "phase2_file": "/kaggle/working/output/phase2_references_XNQTWZ87K4I.json",  # ⚠️ UPDATE
    
    # Frame extraction
    "frame_offsets_seconds": [-1, 0, 1, 2],  # Extract at -1s, 0s, +1s, +2s from reference
    "max_frames_per_reference": 4,
    
    # Duplicate detection
    "enable_dedup": True,
    "perceptual_hash_threshold": 5,  # Hamming distance (0-64, lower=stricter)
    
    # Quality filtering
    "min_brightness": 20,   # Skip very dark frames
    "max_brightness": 250,  # Skip very bright/washed out frames
    "min_sharpness": 50,    # Skip blurry frames
    
    # OCR settings
    "enable_ocr": False,  # Set True to enable
    "ocr_engine": "tesseract",  # "tesseract" or "paddle"
    "ocr_languages": "eng",  # Language codes
    
    # AI description (Gemini)
    "enable_ai_description": True if gemini_api_key else False,  # Auto-enable if key exists
    "gemini_api_key": gemini_api_key,  # Loaded from Kaggle secrets
    "gemini_model": "gemini-2.5-flash",
    
    # Output
    "output_dir": "/kaggle/working/output",
    "frames_dir": "/kaggle/working/frames",
    "save_thumbnails": True,  # Save smaller versions
    "thumbnail_size": (640, 360)
}

# Create directories
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['frames_dir'], exist_ok=True)

print("✅ Configuration loaded")
print(f"   Frame offsets: {CONFIG['frame_offsets_seconds']}")
print(f"   Deduplication: {'Enabled' if CONFIG['enable_dedup'] else 'Disabled'}")
print(f"   OCR: {'Enabled (' + CONFIG['ocr_engine'] + ')' if CONFIG['enable_ocr'] else 'Disabled'}")
print(f"   AI Description: {'Enabled' if CONFIG['enable_ai_description'] else 'Disabled'}")

if CONFIG['enable_ai_description']:
    print(f"   🔑 API key ready (length: {len(gemini_api_key)} chars)")

In [ ]:
# ====================================================================
# CELL 4: Load Phase 1 & 2 Data
# ====================================================================

print("📂 Loading previous phase outputs...")

# Load Phase 1 (transcript)
with open(CONFIG['phase1_file'], 'r', encoding='utf-8') as f:
    phase1_data = json.load(f)

# Load Phase 2 (references)
with open(CONFIG['phase2_file'], 'r', encoding='utf-8') as f:
    phase2_data = json.load(f)

video_path = phase1_data['video_path']
video_id = phase1_data['video_id']
references = phase2_data['references']

print(f"✅ Data loaded:")
print(f"   Video: {Path(video_path).name}")
print(f"   Video ID: {video_id}")
print(f"   References: {len(references)}")

if len(references) == 0:
    print("\n⚠️  No references found! Phase 2 didn't detect any visual references.")
    print("   Try lowering the similarity threshold in Phase 2.")
else:
    print(f"\n📍 First reference: {references[0]['timestamp_ms']/1000:.1f}s")
    print(f"   Text: {references[0]['text'][:100]}...")

In [ ]:
# ====================================================================
# CELL 5: Frame Extraction Functions
# ====================================================================

class FrameExtractor:
    """Extract frames from video at specified timestamps."""
    
    def __init__(self, video_path: str):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if not self.cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")
    
    def extract_frame_at_timestamp(self, timestamp_ms: int, output_path: str) -> bool:
        """Extract single frame at timestamp."""
        # Convert timestamp to frame number
        frame_num = int((timestamp_ms / 1000.0) * self.fps)
        
        # Validate frame number
        if frame_num < 0 or frame_num >= self.total_frames:
            return False
        
        # Seek to frame
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.cap.read()
        
        if not ret:
            return False
        
        # Save frame
        cv2.imwrite(output_path, frame)
        return True
    
    def extract_frames_with_offsets(self, base_timestamp_ms: int, 
                                    offsets_seconds: List[float],
                                    output_dir: str,
                                    base_name: str) -> List[Dict[str, Any]]:
        """Extract multiple frames around a timestamp."""
        frames = []
        
        for i, offset in enumerate(offsets_seconds):
            timestamp_ms = base_timestamp_ms + int(offset * 1000)
            output_path = os.path.join(output_dir, f"{base_name}_offset_{offset:+.1f}s.jpg")
            
            success = self.extract_frame_at_timestamp(timestamp_ms, output_path)
            
            if success:
                frames.append({
                    "path": output_path,
                    "timestamp_ms": timestamp_ms,
                    "offset_seconds": offset
                })
        
        return frames
    
    def close(self):
        """Release video capture."""
        self.cap.release()

print("✅ Frame extractor defined")

In [ ]:
# ====================================================================
# CELL 6: Quality Assessment & Deduplication
# ====================================================================

def assess_frame_quality(image_path: str) -> Dict[str, Any]:
    """Assess frame quality (brightness, sharpness)."""
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Brightness (mean pixel value)
    brightness = np.mean(gray)
    
    # Sharpness (Laplacian variance)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    sharpness = laplacian.var()
    
    # Overall quality score (0-1)
    brightness_score = 1 - abs(brightness - 127.5) / 127.5  # Prefer mid-range
    sharpness_score = min(sharpness / 500, 1.0)  # Normalize
    quality_score = (brightness_score * 0.3 + sharpness_score * 0.7)
    
    return {
        "brightness": float(brightness),
        "sharpness": float(sharpness),
        "quality_score": float(quality_score),
        "is_good_quality": (
            CONFIG['min_brightness'] < brightness < CONFIG['max_brightness'] and
            sharpness > CONFIG['min_sharpness']
        )
    }

def calculate_perceptual_hash(image_path: str) -> str:
    """Calculate perceptual hash for duplicate detection."""
    img = Image.open(image_path)
    phash = imagehash.phash(img)
    return str(phash)

def is_duplicate(phash: str, seen_hashes: List[str], threshold: int = 5) -> bool:
    """Check if frame is duplicate based on perceptual hash."""
    for seen_hash in seen_hashes:
        hash1 = imagehash.hex_to_hash(phash)
        hash2 = imagehash.hex_to_hash(seen_hash)
        distance = hash1 - hash2
        
        if distance <= threshold:
            return True
    
    return False

def create_thumbnail(image_path: str, output_path: str, size: Tuple[int, int]):
    """Create thumbnail of image."""
    img = Image.open(image_path)
    img.thumbnail(size, Image.Resampling.LANCZOS)
    img.save(output_path, "JPEG", quality=85)

print("✅ Quality assessment functions defined")

In [ ]:
# ====================================================================
# CELL 7: OCR Functions (Optional)
# ====================================================================

def extract_text_tesseract(image_path: str, lang: str = "eng") -> Dict[str, Any]:
    """Extract text using Tesseract OCR."""
    if not TESSERACT_AVAILABLE:
        return {"text": "", "error": "Tesseract not available"}
    
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img, lang=lang)
        confidence = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
        
        # Calculate average confidence
        confs = [c for c in confidence['conf'] if c != -1]
        avg_conf = np.mean(confs) if confs else 0
        
        return {
            "text": text.strip(),
            "confidence": float(avg_conf),
            "engine": "tesseract"
        }
    except Exception as e:
        return {"text": "", "error": str(e)}

def extract_text_paddle(image_path: str, lang: str = "en") -> Dict[str, Any]:
    """Extract text using PaddleOCR (better for diagrams/equations)."""
    if not PADDLE_AVAILABLE:
        return {"text": "", "error": "PaddleOCR not available"}
    
    try:
        ocr = PaddleOCR(use_angle_cls=True, lang=lang, show_log=False)
        result = ocr.ocr(image_path, cls=True)
        
        # Extract text and confidence
        texts = []
        confidences = []
        
        if result and result[0]:
            for line in result[0]:
                texts.append(line[1][0])
                confidences.append(line[1][1])
        
        return {
            "text": "\n".join(texts),
            "confidence": float(np.mean(confidences)) if confidences else 0,
            "engine": "paddle"
        }
    except Exception as e:
        return {"text": "", "error": str(e)}

def extract_text_from_frame(image_path: str) -> Dict[str, Any]:
    """Extract text using configured OCR engine."""
    if not CONFIG['enable_ocr']:
        return None
    
    if CONFIG['ocr_engine'] == 'paddle':
        return extract_text_paddle(image_path, CONFIG['ocr_languages'])
    else:
        return extract_text_tesseract(image_path, CONFIG['ocr_languages'])

print("✅ OCR functions defined")

In [ ]:
# ====================================================================
# CELL 8: AI Description (Gemini - Optional)
# ====================================================================

def generate_ai_description(image_path: str) -> Optional[Dict[str, Any]]:
    """Generate AI description using Google Gemini."""
    if not CONFIG['enable_ai_description']:
        return None
    
    if not GEMINI_AVAILABLE:
        return {"description": "", "error": "Gemini not available"}
    
    try:
        # Configure API
        api_key = CONFIG['gemini_api_key'] or os.getenv('GEMINI_API_KEY')
        if not api_key:
            return {"description": "", "error": "No API key provided"}
        
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(CONFIG['gemini_model'])
        
        # Load image
        img = Image.open(image_path)
        
        # Generate description
        prompt = """
        Analyze this video frame from a software compliance walkthrough and provide:

        1. A concise description of the UI state (e.g., "Showing the cookie banner settings" or "Inside the admin dashboard").
        2. Identify specific interactive elements visible: (e.g., "Accept/Decline buttons", "Consent tracking table", or "The trash bag widget").
        3. Determine the demonstration context: Is the speaker explaining a concept, showing a specific setting, or navigating to a new menu?

        Format as JSON: 
        {
          "description": "...", 
          "visual_elements": ["element1", "element2"],
          "type": "UI_Walkthrough/Navigation/Data_Table/Settings", 
          "concept": "GDPR Compliance / Cookie Management"
        }
        """
        
        response = model.generate_content([prompt, img])
        
        # Try to parse JSON response
        try:
            result = json.loads(response.text)
        except:
            result = {
                "description": response.text,
                "type": "unknown",
                "concept": ""
            }
        
        return result
        
    except Exception as e:
        return {"description": "", "error": str(e)}

print("✅ AI description function defined")
if CONFIG['enable_ai_description'] and not CONFIG['gemini_api_key']:
    print("⚠️  To use Gemini, set CONFIG['gemini_api_key'] or GEMINI_API_KEY env var")

In [ ]:
# ====================================================================
# CELL 9: Main Processing Loop
# ====================================================================

print("="*70)
print("🚀 STARTING PHASE 3: VISUAL EXTRACTION")
print("="*70)

# Initialize extractor
extractor = FrameExtractor(video_path)

all_frames = []
seen_hashes = []
total_extracted = 0
duplicates_skipped = 0
low_quality_skipped = 0

print(f"\nProcessing {len(references)} references...\n")

for ref_idx, reference in enumerate(references, 1):
    ref_id = reference['reference_id']
    timestamp_ms = reference['timestamp_ms']
    
    print(f"[{ref_idx}/{len(references)}] {ref_id} @ {timestamp_ms/1000:.1f}s")
    print(f"  Text: {reference['text'][:80]}...")
    
    # Create reference directory
    ref_dir = os.path.join(CONFIG['frames_dir'], ref_id)
    os.makedirs(ref_dir, exist_ok=True)
    
    # Extract frames with offsets
    frames = extractor.extract_frames_with_offsets(
        timestamp_ms,
        CONFIG['frame_offsets_seconds'],
        ref_dir,
        ref_id
    )
    
    print(f"  Extracted {len(frames)} frames")
    
    # Process each frame
    for frame in frames:
        total_extracted += 1
        frame_path = frame['path']
        
        # Quality assessment
        quality = assess_frame_quality(frame_path)
        
        if not quality['is_good_quality']:
            print(f"    ⚠️  Low quality (brightness={quality['brightness']:.0f}, sharpness={quality['sharpness']:.0f})")
            low_quality_skipped += 1
            continue
        
        # Duplicate detection
        phash = calculate_perceptual_hash(frame_path)
        is_dup = is_duplicate(phash, seen_hashes, CONFIG['perceptual_hash_threshold']) if CONFIG['enable_dedup'] else False
        
        if is_dup:
            print(f"    🔄 Duplicate detected (skipping)")
            duplicates_skipped += 1
            continue
        
        seen_hashes.append(phash)
        
        # Create thumbnail
        if CONFIG['save_thumbnails']:
            thumb_path = frame_path.replace('.jpg', '_thumb.jpg')
            create_thumbnail(frame_path, thumb_path, CONFIG['thumbnail_size'])
        else:
            thumb_path = None
        
        # OCR extraction
        ocr_data = extract_text_from_frame(frame_path)
        if ocr_data and ocr_data.get('text'):
            print(f"    📝 OCR: {ocr_data['text'][:60]}...")
        
        # AI description
        ai_desc = generate_ai_description(frame_path)
        if ai_desc and ai_desc.get('description'):
            print(f"    🤖 AI: {ai_desc['description'][:60]}...")
        
        # Store frame info
        frame_info = {
            "frame_id": f"{ref_id}_F{len(all_frames)}",
            "reference_id": ref_id,
            "reference_text": reference['text'],
            "timestamp_ms": frame['timestamp_ms'],
            "offset_seconds": frame['offset_seconds'],
            "frame_path": frame_path,
            "thumbnail_path": thumb_path,
            "perceptual_hash": phash,
            "quality": quality,
            "ocr_data": ocr_data,
            "ai_description": ai_desc
        }
        
        all_frames.append(frame_info)
        print(f"    ✅ Processed (quality={quality['quality_score']:.2f})")
    
    print()

# Cleanup
extractor.close()

print("="*70)
print("✅ EXTRACTION COMPLETE")
print("="*70)
print(f"📊 Statistics:")
print(f"   Total extracted: {total_extracted}")
print(f"   Duplicates skipped: {duplicates_skipped}")
print(f"   Low quality skipped: {low_quality_skipped}")
print(f"   Unique frames kept: {len(all_frames)}")
print("="*70)

In [ ]:
# ====================================================================
# CELL 10: Save Results
# ====================================================================

def convert_to_serializable(obj):
    """Convert numpy/non-serializable types to JSON-compatible types."""
    import numpy as np
    
    if isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Convert result to JSON-serializable format
result = {
    "video_id": video_id,
    "video_path": video_path,
    "frames": convert_to_serializable(all_frames),
    "frame_count": len(all_frames),
    "statistics": {
        "total_extracted": int(total_extracted),
        "duplicates_skipped": int(duplicates_skipped),
        "low_quality_skipped": int(low_quality_skipped),
        "unique_frames": len(all_frames)
    },
    "config": {
        "frame_offsets": CONFIG['frame_offsets_seconds'],
        "dedup_enabled": bool(CONFIG['enable_dedup']),
        "ocr_enabled": bool(CONFIG['enable_ocr']),
        "ai_enabled": bool(CONFIG['enable_ai_description'])
    }
}

# Save JSON
output_file = f"{CONFIG['output_dir']}/phase3_frames_{video_id}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Save text report
report_file = f"{CONFIG['output_dir']}/phase3_report_{video_id}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("Visual Frame Extraction Report\n")
    f.write(f"Video ID: {video_id}\n")
    f.write("="*70 + "\n\n")
    
    for i, frame in enumerate(all_frames, 1):
        f.write(f"[{i}] Frame ID: {frame['frame_id']}\n")
        f.write(f"    Time: {frame['timestamp_ms']/1000:.1f}s (offset: {frame['offset_seconds']:+.1f}s)\n")
        f.write(f"    Reference: {frame['reference_text'][:80]}...\n")
        f.write(f"    Quality: {frame['quality']['quality_score']:.2f}\n")
        f.write(f"    Path: {frame['frame_path']}\n")
        
        if frame.get('ocr_data') and frame['ocr_data'].get('text'):
            f.write(f"    OCR: {frame['ocr_data']['text'][:100]}...\n")
        
        if frame.get('ai_description') and frame['ai_description'].get('description'):
            f.write(f"    AI: {frame['ai_description']['description'][:100]}...\n")
        
        f.write("\n")

file_size = os.path.getsize(output_file) / 1024

print("\n💾 Files saved:")
print(f"   JSON: {output_file} ({file_size:.1f} KB)")
print(f"   Report: {report_file}")
print(f"   Frames: {CONFIG['frames_dir']}/")
print("\n✅ Phase 3 complete!")

In [ ]:
# ====================================================================
# CELL 11: Display Sample Frames
# ====================================================================

import matplotlib.pyplot as plt
from matplotlib import gridspec

if len(all_frames) > 0:
    # Show first 6 frames
    num_samples = min(6, len(all_frames))
    
    fig = plt.figure(figsize=(16, 8))
    gs = gridspec.GridSpec(2, 3, hspace=0.3, wspace=0.2)
    
    for i in range(num_samples):
        frame = all_frames[i]
        
        ax = fig.add_subplot(gs[i])
        
        # Load and display image
        img = cv2.imread(frame['frame_path'])
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img_rgb)
        
        # Title with info
        time = frame['timestamp_ms'] / 1000
        quality = frame['quality']['quality_score']
        title = f"{frame['frame_id']}\n{time:.1f}s | Q={quality:.2f}"
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    
    plt.suptitle(f"Sample Extracted Frames ({num_samples}/{len(all_frames)})", 
                 fontsize=14, fontweight='bold')
    plt.savefig(f"{CONFIG['output_dir']}/phase3_samples_{video_id}.png", 
                dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📸 Sample visualization saved!")
else:
    print("⚠️  No frames to display")

In [ ]:
# ====================================================================
# CELL 12: Timeline Visualization
# ====================================================================

if len(all_frames) > 0:
    fig, ax = plt.subplots(figsize=(16, 4))
    
    # Plot frames on timeline
    timestamps = [f['timestamp_ms'] / 1000 for f in all_frames]
    qualities = [f['quality']['quality_score'] for f in all_frames]
    
    scatter = ax.scatter(timestamps, qualities, 
                        s=100, alpha=0.6, c=qualities, 
                        cmap='viridis', edgecolors='black', linewidth=0.5)
    
    # Add reference markers
    ref_times = [r['timestamp_ms'] / 1000 for r in references]
    for t in ref_times:
        ax.axvline(x=t, color='red', alpha=0.3, linestyle='--', linewidth=1)
    
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Frame Quality Score', fontsize=12)
    ax.set_title('Extracted Frames Timeline\n(Red lines = reference timestamps)', 
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.colorbar(scatter, label='Quality Score', ax=ax)
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/phase3_timeline_{video_id}.png", 
                dpi=150, bbox_inches='tight')
    plt.show()
    
    print("📊 Timeline visualization saved!")

print("\n✅ All visualizations complete!")
print(f"\n📁 Download all files from: {CONFIG['output_dir']}/")

**Phase 4**

In [ ]:
# ====================================================================
# CELL 1: Install Dependencies
# ====================================================================
print("📦 Installing dependencies...")

# Core dependencies
!pip install -q transformers torch pillow scikit-learn numpy

# CLIP model
!pip install -q sentence-transformers  # Includes CLIP

print("✅ Dependencies installed!")

In [ ]:
# ====================================================================
# CELL 2: Import Libraries
# ====================================================================
import os
import json
import numpy as np
from typing import Dict, List, Any, Tuple
from collections import defaultdict
from pathlib import Path

import torch
from PIL import Image
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import gc

print(f"🖥️  Device: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

print("✅ Libraries imported")

In [ ]:
# ====================================================================
# CELL 3: Configuration
# ====================================================================

CONFIG = {
    # Input files from previous phases
    "phase1_file": "/kaggle/working/output/transcript_XNQTWZ87K4I.json",  # ⚠️ UPDATE
    "phase2_file": "/kaggle/working/output/phase2_references_XNQTWZ87K4I.json",  # ⚠️ UPDATE
    "phase3_file": "/kaggle/working/output/phase3_frames_XNQTWZ87K4I.json",  # ⚠️ UPDATE
    
    # CLIP model for visual similarity
    "clip_model": "clip-ViT-B-32",  # Options: clip-ViT-B-32, clip-ViT-L-14 (larger, better)
    
    # Clustering parameters (DBSCAN)
    "clustering": {
        "eps": 0.3,  # Distance threshold (0.2-0.4, lower=stricter)
        "min_samples": 1,  # Minimum frames to form cluster
        "metric": "cosine"  # Cosine distance for CLIP embeddings
    },
    
    # Representative selection
    "select_best_by": "quality",  # "quality" or "central" (most similar to cluster center)
    
    # Output
    "output_dir": "/kaggle/working/output",
    "final_output_file": "enriched_transcript.json",
    "save_intermediate": True
}

os.makedirs(CONFIG['output_dir'], exist_ok=True)

print("✅ Configuration loaded")
print(f"   CLIP model: {CONFIG['clip_model']}")
print(f"   DBSCAN eps: {CONFIG['clustering']['eps']}")
print(f"   Min samples: {CONFIG['clustering']['min_samples']}")

In [ ]:
# ====================================================================
# CELL 4: Load All Previous Phase Data
# ====================================================================

print("📂 Loading Phase 1, 2, 3 outputs...\n")

# Load Phase 1 (transcript)
with open(CONFIG['phase1_file'], 'r', encoding='utf-8') as f:
    phase1_data = json.load(f)
print(f"✅ Phase 1: {len(phase1_data['utterances'])} utterances")

# Load Phase 2 (references)
with open(CONFIG['phase2_file'], 'r', encoding='utf-8') as f:
    phase2_data = json.load(f)
print(f"✅ Phase 2: {len(phase2_data['references'])} references")

# Load Phase 3 (frames)
with open(CONFIG['phase3_file'], 'r', encoding='utf-8') as f:
    phase3_data = json.load(f)
print(f"✅ Phase 3: {len(phase3_data['frames'])} frames")

video_id = phase1_data['video_id']
frames = phase3_data['frames']

# Filter valid frames (with existing paths)
valid_frames = [
    f for f in frames 
    if f.get('frame_path') and os.path.exists(f['frame_path'])
]

print(f"\n📊 Summary:")
print(f"   Video ID: {video_id}")
print(f"   Total frames: {len(frames)}")
print(f"   Valid frames: {len(valid_frames)}")

if len(valid_frames) == 0:
    print("\n⚠️  No valid frames found! Check Phase 3 output.")

In [ ]:
# ====================================================================
# CELL 5: CLIP Embedding Functions
# ====================================================================

class CLIPEmbedder:
    """Generate CLIP embeddings for images."""
    
    def __init__(self, model_name: str = "clip-ViT-B-32"):
        print(f"🤖 Loading CLIP model: {model_name}...")
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(model_name, device=device)
        self.device = device
        print(f"✅ CLIP model loaded on {device}")
    
    def embed_image(self, image_path: str) -> np.ndarray:
        """Generate embedding for single image."""
        try:
            img = Image.open(image_path).convert('RGB')
            embedding = self.model.encode(img, convert_to_numpy=True)
            return embedding
        except Exception as e:
            print(f"⚠️  Error encoding {image_path}: {e}")
            return None
    
    def embed_images_batch(self, image_paths: List[str], batch_size: int = 8) -> np.ndarray:
        """Generate embeddings for multiple images in batches."""
        embeddings = []
        
        print(f"   Encoding {len(image_paths)} images in batches of {batch_size}...")
        
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            
            # Load images
            images = []
            for path in batch_paths:
                try:
                    img = Image.open(path).convert('RGB')
                    images.append(img)
                except Exception as e:
                    print(f"⚠️  Skipping {path}: {e}")
                    images.append(None)
            
            # Encode batch
            valid_images = [img for img in images if img is not None]
            if valid_images:
                batch_embeddings = self.model.encode(
                    valid_images,
                    convert_to_numpy=True,
                    show_progress_bar=False
                )
                embeddings.extend(batch_embeddings)
            
            if (i // batch_size + 1) % 10 == 0:
                print(f"   Progress: {i+len(batch_paths)}/{len(image_paths)}")
        
        return np.array(embeddings)

print("✅ CLIP embedder class defined")

In [ ]:
# ====================================================================
# CELL 6: Clustering & Selection Functions
# ====================================================================

def cluster_embeddings(embeddings: np.ndarray, eps: float = 0.3, 
                      min_samples: int = 1, metric: str = 'cosine') -> np.ndarray:
    """Cluster embeddings using DBSCAN."""
    print(f"\n🔍 Clustering {len(embeddings)} embeddings...")
    print(f"   eps={eps}, min_samples={min_samples}, metric={metric}")
    
    # DBSCAN clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
    labels = clustering.fit_predict(embeddings)
    
    # Count clusters
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    
    print(f"✅ Found {n_clusters} clusters + {n_noise} noise points")
    
    return labels

def select_best_representative(frames: List[Dict], frame_indices: List[int],
                              embeddings: np.ndarray, method: str = "quality") -> int:
    """Select best representative frame from cluster."""
    if len(frame_indices) == 1:
        return frame_indices[0]
    
    if method == "quality":
        # Select frame with highest quality score
        qualities = [
            frames[idx].get('quality', {}).get('quality_score', 0)
            for idx in frame_indices
        ]
        best_local_idx = np.argmax(qualities)
        return frame_indices[best_local_idx]
    
    elif method == "central":
        # Select frame closest to cluster centroid
        cluster_embeddings = embeddings[frame_indices]
        centroid = cluster_embeddings.mean(axis=0)
        
        # Find closest to centroid
        similarities = cosine_similarity([centroid], cluster_embeddings)[0]
        best_local_idx = np.argmax(similarities)
        return frame_indices[best_local_idx]
    
    return frame_indices[0]

print("✅ Clustering functions defined")

In [ ]:
# ====================================================================
# CELL 7: Generate CLIP Embeddings
# ====================================================================

if len(valid_frames) > 0:
    print("="*70)
    print("🚀 GENERATING CLIP EMBEDDINGS")
    print("="*70)
    
    # Initialize embedder
    embedder = CLIPEmbedder(CONFIG['clip_model'])
    
    # Extract frame paths
    frame_paths = [f['frame_path'] for f in valid_frames]
    
    # Generate embeddings
    embeddings = embedder.embed_images_batch(frame_paths, batch_size=8)
    
    print(f"\n✅ Generated {len(embeddings)} embeddings")
    print(f"   Embedding dimension: {embeddings.shape[1]}")
    
    # Clean up
    del embedder
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
else:
    print("⚠️  No valid frames to embed")
    embeddings = np.array([])

In [ ]:
# ====================================================================
# CELL 8: Cluster Similar Frames
# ====================================================================

if len(embeddings) > 0:
    print("\n" + "="*70)
    print("🔍 CLUSTERING VISUAL FRAMES")
    print("="*70)
    
    # Cluster embeddings
    labels = cluster_embeddings(
        embeddings,
        eps=CONFIG['clustering']['eps'],
        min_samples=CONFIG['clustering']['min_samples'],
        metric=CONFIG['clustering']['metric']
    )
    
    # Organize clusters
    clusters = defaultdict(list)
    frame_to_cluster = {}
    
    for i, (frame, label) in enumerate(zip(valid_frames, labels)):
        frame_id = frame['frame_id']
        
        if label == -1:
            # Noise point - unique visual
            cluster_id = f"VISUAL_{i:03d}_SINGLE"
        else:
            # Part of cluster
            cluster_id = f"VISUAL_{label:03d}"
        
        clusters[cluster_id].append(i)
        frame_to_cluster[frame_id] = cluster_id
    
    print(f"\n📊 Clustering results:")
    print(f"   Total clusters: {len(clusters)}")
    print(f"   Average cluster size: {np.mean([len(v) for v in clusters.values()]):.1f}")
    print(f"   Largest cluster: {max(len(v) for v in clusters.values())} frames")
    
    # Select representatives
    print(f"\n🎯 Selecting best representatives (method: {CONFIG['select_best_by']})...")
    cluster_representatives = {}
    
    for cluster_id, frame_indices in clusters.items():
        best_idx = select_best_representative(
            valid_frames,
            frame_indices,
            embeddings,
            method=CONFIG['select_best_by']
        )
        cluster_representatives[cluster_id] = best_idx
    
    print(f"✅ Selected {len(cluster_representatives)} representatives")
    
else:
    print("⚠️  No embeddings to cluster")
    clusters = {}
    frame_to_cluster = {}
    cluster_representatives = {}
    labels = np.array([])

In [ ]:
# ====================================================================
# CELL 9: Create Visual Database
# ====================================================================

print("\n" + "="*70)
print("🎨 CREATING VISUAL DATABASE")
print("="*70)

visuals = {}

for cluster_id, frame_indices in clusters.items():
    if not frame_indices:
        continue
    
    # Get representative frame
    rep_idx = cluster_representatives.get(cluster_id, frame_indices[0])
    rep_frame = valid_frames[rep_idx]
    
    # Collect all timestamps where this visual appears
    timestamps = [valid_frames[idx]['timestamp_ms'] for idx in frame_indices]
    
    # Build visual entry
    visual_entry = {
        "representative_frame": rep_frame['frame_path'],
        "thumbnail": rep_frame.get('thumbnail_path'),
        "appears_at_ms": sorted(timestamps),
        "appearance_count": len(timestamps),
        "quality": rep_frame.get('quality', {}),
        "frame_ids": [valid_frames[idx]['frame_id'] for idx in frame_indices]
    }
    
    # Add OCR data if available
    ocr_data = rep_frame.get('ocr_data')
    if ocr_data and ocr_data.get('text'):
        visual_entry['ocr_text'] = ocr_data['text']
        visual_entry['ocr_confidence'] = ocr_data.get('confidence', 0)
    
    # Add AI description if available
    ai_desc = rep_frame.get('ai_description')
    if ai_desc:
        visual_entry['description'] = ai_desc.get('description', '')
        visual_entry['visual_type'] = ai_desc.get('type', 'unknown')
        visual_entry['concept'] = ai_desc.get('concept', '')
    
    visuals[cluster_id] = visual_entry

print(f"✅ Created visual database with {len(visuals)} unique visuals")

# Show distribution
multi_appearance = sum(1 for v in visuals.values() if v['appearance_count'] > 1)
print(f"   Visuals appearing once: {len(visuals) - multi_appearance}")
print(f"   Visuals appearing multiple times: {multi_appearance}")

In [ ]:
# ====================================================================
# CELL 10: Create Enriched Transcript
# ====================================================================

print("\n" + "="*70)
print("📝 CREATING ENRICHED TRANSCRIPT")
print("="*70)

# Build mapping from reference to cluster
reference_to_visual = {}

for frame in valid_frames:
    frame_id = frame['frame_id']
    reference_id = frame['reference_id']
    
    if frame_id in frame_to_cluster:
        cluster_id = frame_to_cluster[frame_id]
        
        # Map reference to visual (use first occurrence)
        if reference_id not in reference_to_visual:
            reference_to_visual[reference_id] = cluster_id

# Create transcript entries
transcript_entries = []
references = phase2_data.get('references', [])

for reference in references:
    reference_id = reference['reference_id']
    visual_id = reference_to_visual.get(reference_id)
    
    entry = {
        "timestamp_ms": reference['timestamp_ms'],
        "text": reference['text'],
        "visual_id": visual_id,
        "reference_type": reference.get('reference_type', 'unknown'),
        "detection_method": reference.get('detection_method', 'unknown')
    }
    
    # Add visual metadata if available
    if visual_id and visual_id in visuals:
        visual = visuals[visual_id]
        entry['visual_description'] = visual.get('description')
        entry['visual_type'] = visual.get('visual_type')
        entry['ocr_text'] = visual.get('ocr_text')
    
    transcript_entries.append(entry)

print(f"✅ Created {len(transcript_entries)} transcript entries")
print(f"   Entries with visuals: {sum(1 for e in transcript_entries if e['visual_id'])}")

In [ ]:
# ====================================================================
# CELL 11: Save Final Results
# ====================================================================

# Create final enriched transcript
enriched_transcript = {
    "video_id": video_id,
    "video_url": phase1_data.get('video_url', ''),
    "video_title": phase1_data.get('video_title', ''),
    "duration_ms": phase1_data['duration_ms'],
    
    # Main outputs
    "transcript": transcript_entries,
    "visuals": visuals,
    
    # Metadata
    "metadata": {
        "total_utterances": len(phase1_data['utterances']),
        "total_references": len(references),
        "total_frames_extracted": len(frames),
        "valid_frames": len(valid_frames),
        "unique_visuals": len(visuals),
        "clip_model": CONFIG['clip_model'],
        "clustering_eps": CONFIG['clustering']['eps']
    }
}

# Save main output
output_file = os.path.join(CONFIG['output_dir'], CONFIG['final_output_file'])
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(enriched_transcript, f, ensure_ascii=False, indent=2)

# Save clustering details (intermediate)
if CONFIG['save_intermediate']:
    clustering_file = f"{CONFIG['output_dir']}/phase4_clustering_{video_id}.json"
    clustering_data = {
        "clusters": {k: v for k, v in clusters.items()},
        "frame_to_cluster": frame_to_cluster,
        "cluster_representatives": cluster_representatives,
        "cluster_sizes": {k: len(v) for k, v in clusters.items()}
    }
    with open(clustering_file, 'w', encoding='utf-8') as f:
        json.dump(clustering_data, f, ensure_ascii=False, indent=2)
    print(f"💾 Clustering details: {clustering_file}")

# Save human-readable report
report_file = f"{CONFIG['output_dir']}/phase4_report_{video_id}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("ENRICHED TRANSCRIPT REPORT\n")
    f.write(f"Video ID: {video_id}\n")
    f.write("="*70 + "\n\n")
    
    f.write("VISUAL DATABASE\n")
    f.write("-"*70 + "\n")
    for visual_id, visual in list(visuals.items())[:20]:  # First 20
        f.write(f"\n{visual_id}:\n")
        f.write(f"  Appears: {visual['appearance_count']} times\n")
        f.write(f"  Times: {', '.join([str(t/1000) + 's' for t in visual['appears_at_ms'][:5]])}...\n")
        if visual.get('description'):
            f.write(f"  Description: {visual['description'][:100]}...\n")
        if visual.get('ocr_text'):
            f.write(f"  OCR: {visual['ocr_text'][:100]}...\n")
    
    if len(visuals) > 20:
        f.write(f"\n... and {len(visuals) - 20} more visuals\n")
    
    f.write("\n" + "="*70 + "\n")
    f.write("\nTRANSCRIPT ENTRIES\n")
    f.write("-"*70 + "\n")
    for i, entry in enumerate(transcript_entries[:10], 1):  # First 10
        f.write(f"\n[{i}] {entry['timestamp_ms']/1000:.1f}s\n")
        f.write(f"  Text: {entry['text'][:100]}...\n")
        f.write(f"  Visual: {entry['visual_id'] or 'None'}\n")
        if entry.get('visual_description'):
            f.write(f"  Description: {entry['visual_description'][:80]}...\n")

file_size = os.path.getsize(output_file) / 1024

print("\n" + "="*70)
print("✅ PHASE 4 COMPLETE - PIPELINE FINISHED!")
print("="*70)
print(f"📊 Final Statistics:")
print(f"   Video ID: {video_id}")
print(f"   Duration: {enriched_transcript['duration_ms']/1000:.1f}s")
print(f"   Total references: {enriched_transcript['metadata']['total_references']}")
print(f"   Unique visuals: {enriched_transcript['metadata']['unique_visuals']}")
print(f"   Frames analyzed: {enriched_transcript['metadata']['total_frames_extracted']}")
print(f"\n💾 Output files:")
print(f"   Main: {output_file} ({file_size:.1f} KB)")
print(f"   Report: {report_file}")
print("="*70)

**Integration**

In [ ]:
import json
import os

# Define the output directory
OUTPUT_DIR = "/kaggle/working/output"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 1. File Paths (based on your uploads)
phase1_file = '/kaggle/working/output/transcript_XNQTWZ87K4I.json' 
enriched_file = '/kaggle/working/output/enriched_transcript.json'

# 2. Load Data
with open(phase1_file, 'r', encoding='utf-8') as f:
    phase1_data = json.load(f)

with open(enriched_file, 'r', encoding='utf-8') as f:
    enriched_data = json.load(f)

# The fix: 'visuals' in your file is a dict mapping ID -> Data
visual_lookup = enriched_data.get('visuals', {})

# 3. Create a sorted timeline of visual detection points
visual_timeline = []
for entry in enriched_data.get('transcript', []):
    if entry.get('visual_id'):
        visual_timeline.append({
            'ts': entry['timestamp_ms'],
            'id': entry['visual_id']
        })
visual_timeline = sorted(visual_timeline, key=lambda x: x['ts'])

def find_active_visual(start_ms, end_ms):
    """Finds which visual state was active during this speech segment."""
    # Find a visual that was triggered during this specific sentence
    for event in visual_timeline:
        if start_ms <= event['ts'] <= end_ms:
            return event['id']
    
    # Fallback: Find the most recent visual state set before this sentence
    active_id = None
    for event in visual_timeline:
        if event['ts'] <= start_ms:
            active_id = event['id']
        else:
            break
    return active_id

# 4. Perform the Join
final_entries = []
for utt in phase1_data.get('utterances', []):
    v_id = find_active_visual(utt['start_ms'], utt['end_ms'])
    v_info = visual_lookup.get(v_id)
    
    # Clean up the AI description if it's wrapped in markdown code blocks
    raw_desc = v_info.get('description', '') if v_info else ""
    clean_desc = raw_desc.replace('```json', '').replace('```', '').strip()
    
    entry = {
        "timestamp_ms": utt['start_ms'],
        "time": f"{utt['start_ms']/1000:.1f}s",
        "text": utt['text'],
        "visual_id": v_id,
        "visual_details": clean_desc,
        "elements_on_screen": v_info.get('visual_elements', []) if v_info else []
    }
    final_entries.append(entry)

# 5. Save the final integrated file
final_output = {
    "video_info": {
        "title": phase1_data.get('video_title'),
        "url": phase1_data.get('video_url')
    },
    "multimodal_transcript": final_entries
}

output_path = os.path.join(OUTPUT_DIR, "FINAL_INTEGRATED_OUTPUT.json")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(final_output, f, ensure_ascii=False, indent=2)

print(f"✅ Integration complete!")
print(f"📂 Final file saved to: {output_path}")

**final transcript**

**phase1+phase4**

In [ ]:
import json
import os

# 1. Setup paths
INPUT_TRANSCRIPT = '/kaggle/working/output/transcript_XNQTWZ87K4I.json'
INPUT_ENRICHED = '/kaggle/working/output/enriched_transcript.json'
OUTPUT_PATH = '/kaggle/working/output/FINAL_NARRATIVE_TRANSCRIPT.txt'

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# 2. Load the data from your phases
with open(INPUT_TRANSCRIPT, 'r', encoding='utf-8') as f:
    phase1 = json.load(f)
with open(INPUT_ENRICHED, 'r', encoding='utf-8') as f:
    enriched = json.load(f)

visual_lookup = enriched.get('visuals', {})
visual_events = sorted(
    [e for e in enriched.get('transcript', []) if e.get('visual_id')],
    key=lambda x: x['timestamp_ms']
)

# 3. Process and Build the Text Transcript
lines = []
lines.append(f"NARRATIVE TRANSCRIPT: {phase1.get('video_title', 'GDPR Video')}")
lines.append("="*80 + "\n")

last_visual_id = None

for utt in phase1.get('utterances', []):
    start_ms = utt['start_ms']
    # Format time as [MM:SS]
    time_str = f"[{int(start_ms//60000):02d}:{int((start_ms%60000)//1000):02d}]"
    
    # Check for visual context update
    current_visual_id = None
    for event in visual_events:
        if event['timestamp_ms'] <= start_ms:
            current_visual_id = event['visual_id']
        else:
            break
    
    # If the visual state changed, insert a descriptive block
    if current_visual_id and current_visual_id != last_visual_id:
        v_info = visual_lookup.get(current_visual_id, {})
        raw_desc = v_info.get('description', '')
        
        # Clean the AI description (removes JSON code blocks)
        try:
            clean_json_str = raw_desc.replace('```json', '').replace('```', '').strip()
            desc_obj = json.loads(clean_json_str)
            main_desc = desc_obj.get('description', '')
            elements = desc_obj.get('visual_elements', [])
        except:
            main_desc = raw_desc
            elements = []

        lines.append(f"\n--- [VISUAL CONTEXT CHANGE] ---")
        lines.append(f"SCENE: {main_desc}")
        if elements:
            lines.append(f"VISIBLE ON SCREEN: {', '.join(elements)}")
        lines.append("-" * 31 + "\n")
        
        last_visual_id = current_visual_id

    # Append the spoken text
    lines.append(f"{time_str} {utt['text']}")

# 4. Write to the final text file
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    f.write("\n".join(lines))

print(f"✅ Readable transcript successfully created at: {OUTPUT_PATH}")

**phase1+phase3**

In [ ]:
import json
import os

# 1. Setup paths
INPUT_TRANSCRIPT = '/kaggle/working/output/transcript_XNQTWZ87K4I.json'
INPUT_PHASE3 = '/kaggle/working/output/phase3_frames_XNQTWZ87K4I.json'
OUTPUT_PATH = '/kaggle/working/output/PHASE3_NARRATIVE_TRANSCRIPT.txt'

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# 2. Load the data
with open(INPUT_TRANSCRIPT, 'r', encoding='utf-8') as f:
    phase1 = json.load(f)
with open(INPUT_PHASE3, 'r', encoding='utf-8') as f:
    phase3 = json.load(f)

# Sort frames by timestamp to ensure chronological order
frames = sorted(phase3.get('frames', []), key=lambda x: x['timestamp_ms'])

# 3. Build the Narrative Text
lines = []
lines.append(f"PHASE 3 INTEGRATED TRANSCRIPT: {phase1.get('video_title', 'GDPR Video')}")
lines.append("="*80 + "\n")

frame_idx = 0
num_frames = len(frames)

for utt in phase1.get('utterances', []):
    start_ms = utt['start_ms']
    end_ms = utt['end_ms']
    time_str = f"[{int(start_ms//60000):02d}:{int((start_ms%60000)//1000):02d}]"
    
    # Insert visual signals that occur before or during this speech segment
    while frame_idx < num_frames and frames[frame_idx]['timestamp_ms'] <= end_ms:
        frame = frames[frame_idx]
        raw_ai = frame.get('ai_description', {})
        raw_desc_str = raw_ai.get('description', '')
        
        # Parse the Vision AI's JSON description
        try:
            clean_json_str = raw_desc_str.replace('```json', '').replace('```', '').strip()
            desc_obj = json.loads(clean_json_str)
            main_desc = desc_obj.get('description', 'No description available.')
            elements = desc_obj.get('visual_elements', [])
            v_type = desc_obj.get('type', 'Unknown')
        except:
            main_desc = raw_desc_str
            elements = []
            v_type = "Unknown"

        lines.append(f"\n[ VISUAL SIGNAL @ {frame['timestamp_ms']/1000:.1f}s | ID: {frame['frame_id']} ]")
        lines.append(f"TYPE: {v_type}")
        lines.append(f"SCENE: {main_desc}")
        if elements:
            lines.append(f"UI ELEMENTS: {', '.join(elements)}")
        lines.append("-" * 50 + "\n")
        
        frame_idx += 1

    # Append the spoken text
    lines.append(f"{time_str} {utt['text']}")

# 4. Save to file
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    f.write("\n".join(lines))

print(f"✅ Phase 3 integrated transcript successfully created at: {OUTPUT_PATH}")